In [35]:
import pandas as pd
import seaborn as sns
import os

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [36]:
os.getcwd()

'/home/antonioskilton/mlops-zoomcamp/01-intro'

In [37]:
df = pd.read_parquet('../data/fhv_tripdata_2021-01.parquet')

In [38]:
old_len = len(df)

In [39]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df['duration'] = df.duration.dt.total_seconds() / 60

In [40]:
df.duration.mean()

19.1672240937939

In [41]:
df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

In [42]:
categorical = ['PUlocationID', 'DOlocationID']

df[categorical] = df[categorical].fillna(-1).astype('int')

In [43]:
df[categorical] = df[categorical].astype('str')

In [44]:
train_dicts = df[categorical].to_dict(orient='records')

In [45]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts) 

In [46]:
X_train.shape

(1109826, 525)

In [47]:
y_train = df.duration.values

In [48]:
len(dv.feature_names_)

525

In [49]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [50]:
y_pred = lr.predict(X_train)

In [51]:
mean_squared_error(y_train, y_pred, squared=False)

10.528519107202367

In [52]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [53]:
df_val = read_data('../data/fhv_tripdata_2021-02.parquet')

In [54]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [55]:
X_val = dv.transform(val_dicts) 

In [56]:
y_pred = lr.predict(X_val)

In [57]:
y_val = df_val.duration.values

In [58]:
mean_squared_error(y_val, y_pred, squared=False)

11.014283131720175